In [2]:
library(viridis)
library(plot3D)
library(stringr)
library(factoextra)
library(FactoMineR)
library(vegan)

setwd("~/mnt/scgc/simon/microg2p/analyses/20210325_GoM_recluster/20210325_GoM_recluster_analysis/O2_consumption")

#read input data
file_name <- "KO_Transcripts_percell_O2_percell_10SAG_GENstats_biogeochemical_master_no2017.csv"
main_title <- "KO_Transcripts_percell_O2_percell_10SAG_GENstats_biogeochemical_master_no2017"
stats_name <- "KO_Transcripts"
print(paste("Reading",file_name))
input_data <- read.csv(file=file_name, header=TRUE)


Loading required package: viridisLite
Warning message:
“no DISPLAY variable so Tk is not available”

ERROR: Error in value[[3L]](cond): Package ‘stringr’ version 1.3.1 cannot be unloaded:
 Error in unloadNamespace(package) : namespace ‘stringr’ is imported by ‘evaluate’ so cannot be unloaded



In [ ]:

print("Cleaning data")
#keep data upto to but not including the Weighted_avg_O2 column
O2_col <- which(str_detect(string=colnames(input_data), 
                      pattern="Weighted_avg_O2", negate = FALSE))
community_data <- input_data[,4:(O2_col-1)]
#replace NA data with zeros
community_data[is.na(community_data)] <- 0

environmental_data_all <- input_data[,O2_col:ncol(input_data)]
environmental_data <- environmental_data_all[,c(1,10,11,13,16,17,18,24,30,31,32,33)]
environmental_data <- scale(environmental_data)



In [ ]:
#RDA analysis
my_rda <- rda(X=community_data, Y=environmental_data, scale=TRUE)
#print contributions to each RDA principle component
cat("\n\nComposition of RDA Principle Components\n")
print(my_rda$CCA$biplot)

#bar plot of variation accounted for
op <- par(no.readonly = TRUE)
eig <- eigenvals(my_rda)
eig_percent <- 100*eig/sum(eig)
par(mar = c(5, 4, 4, 4) + 0.4) 
a_bar <- barplot(eig_percent[1:6], ylim=c(0,3),
        xlab="RDA Principle Axis", 
        ylab="Percent Variation Explained", 
        cex.names=0.7)
print(par("usr"))



In [ ]:
#add plot of cumulative percentage
y_max <- 8
par(new = TRUE) 
ax_lim <- par("usr")
y2 <- mapply(function(i) sum(eig_percent[1:i]), i=1:6)
#y2 <-rep(0,6)
plot(x=seq(from=ax_lim[1]+0.1*(ax_lim[2]-ax_lim[1]), 
           to= ax_lim[1]+1.1*(ax_lim[2]-ax_lim[1]),
           length.out=6),
     y=y2, type="l",
     ylim=c(0,y_max),  xlim=c(0.0,8.8), 
     col = "black", lwd=2, axes = FALSE, xlab = "", ylab = "")
axis(side = 4, at = seq(from=0, to=y_max, by=2))  
mtext("Cumulative Percent", side = 4, line = 2.5)
print(par("usr"))
par <- op



In [ ]:
#tri-plots of RDA1 vs RDA2 with different scaling options
for (scale_type in 0:3){
abc <- plot(x=my_rda, scaling=scale_type,
            xlab="RDA1", ylab="RDA2",
            main=paste("Scaling type:",scale_type), cex.main=0.8)
}

rda_holder <- function(){
  ordiplot(ord=my_rda, choices = c(1,2), type="points", 
           display="sites", scaling=3, xlab="RDA1",ylab="RDA2",
           main="Sites (scaling=3)")
  
  ordiplot(ord=my_rda, choices = c(1,2), type="points", 
           display="species", scaling=3, xlab="RDA1",ylab="RDA2",
           main="Species (scaling=3)")
  
  ordiplot(ord=my_rda, choices = c(1,2), type="points", 
           display="bp", scaling=3, xlab="RDA1",ylab="RDA2",
           main="Biplot (scaling=3)")
  
  site_scores <- scores(x=my_rda,choices=c(1,2), 
                        display="sites", scaling=scale_type)
  species_scores <- scores(x=my_rda, choices=c(1,2), 
                           display="species", scaling=scale_type)
  bp_scores <- scores(x=my_rda, choices=c(1,2), 
                      display="bp", scaling=scale_type)
  
  plot(x=species_scores[,1], y=species_scores[,2],
       xlim=c(-2,3), ylim=c(-1.6,1.6), 
       col="red", main="Species Scores")
  plot(x=site_scores[,1], y=site_scores[,2],
       xlim=c(-2,3), ylim=c(-1.6,1.6), xlab="RDA1", ylab="RDA2",
       col="black", main="Site Scores")
  arrows2D(0,0,15*bp_scores[,1], 15*bp_scores[,2], length=0.1,
           xlim=c(-2,3), ylim=c(-1.6,1.6), xlab="RDA1", ylab="RDA2",
           col="blue", main="Biplot Scores")
  
  const <- attributes(abc$biplot)$const/5
  plot(x=const*abc$biplot[,1], y=const*abc$biplot[,2],
       xlim=c(-2,3), ylim=c(-1.6,1.6), xlab="RDA1", ylab="RDA2",
       col="blue", main="Biplot Scores")
  arrows(0,0,const*abc$biplot[,1],const*abc$biplot[,2], 
         length=0.1)
}




In [ ]:
my_pca <- function(){
  if (norm_or_scale == 1){
    norm_scale <- "raw data"
  }
  
  if (norm_or_scale == 2){
    print("z normalizing data")
    community_data_a <- community_data
    #with what's left, scale and center by ROWS?
    community_data <- t(scale(t(community_data_a), center=TRUE, scale=TRUE))
    norm_scale <- "z norming"
  }
  
  if (norm_or_scale == 3){
    print("mean normalizing data")
    #normalize each row by its mean
    community_data_a <- community_data
    mean_by_row <- rowMeans(community_data_a)
    for (ijk in 1:nrow(community_data_a)){
      community_data[ijk,] <- community_data_a[ijk,]/mean_by_row[ijk]
    }
    norm_scale <- "mean norming"
  }
  print("Conducting PCA")
  gen_pca <- PCA(community_data, scale.unit=TRUE, graph=FALSE)
  
  phyla_keep <- str_detect(string=colnames(input_data), pattern="Phyla", negate = FALSE)
  phyla_col <- which(phyla_keep)
  phyla_unique <- unique(input_data[,phyla_col])
  n_phyla <- length(phyla_unique)
  
  #class_keep <- str_detect(string=colnames(input_data), pattern="Class", negate = FALSE)
  class_col <- phyla_col+1   #which(class_keep)
  class_unique <- unique(input_data[,class_col])
  n_class <- length(class_unique)
  
  #order_keep <- str_detect(string=colnames(input_data), pattern="Order", negate = FALSE)
  order_col <- phyla_col+2 #which(order_keep)
  order_unique <- unique(input_data[,order_col])
  n_order <- length(order_unique)
  
  #family_keep <- str_detect(string=colnames(input_data), pattern="Family", negate = FALSE)
  family_col <- phyla_col+3  #which(family_keep)
  family_unique <- unique(input_data[,family_col])
  n_family <- length(family_unique)
  
  #save original graphic parameter settings
  op <- par(no.readonly = TRUE)
  
  for (j in 1:4){
    sym_var <- rep(NA,times=nrow(input_data))
    classification <- paste("Phyla", norm_scale)
    data_unique <- phyla_unique
    data_col <- phyla_col
    sym_unique <- seq_along(class_unique)
    if (j == 1) {
      sym_name <- input_data[,phyla_col]
      for (k in seq_along(phyla_unique)){
        change <- (sym_name == phyla_unique[k])
        sym_var[change] <- k
      }
    }
    else if (j == 2) {
      sym_name <- input_data[,order_col]
      classification <- paste("Order", norm_scale)
      data_unique <- order_unique
      data_col <- order_col
      sym_unique <- -(64+seq_along(order_unique))
      for (k in seq_along(order_unique)){
        change <- (sym_name == order_unique[k])
        sym_var[change] <- -64-k  #use UNICODE plotting symbols (mostly capital letters)
      }
    }
    else if (j == 3) {
      sym_name <- input_data[,class_col]
      classification <- paste("Class", norm_scale)
      data_unique <- class_unique
      data_col <- class_col
      sym_unique <- seq_along(class_unique)
      for (k in seq_along(class_unique)){
        change <- (sym_name == class_unique[k])
        sym_var[change] <- k
      }
    }
    else if (j == 4) {
      classification <- paste("Family", norm_scale)
      sym_name <- input_data[,family_col]
      data_unique <- family_unique
      data_col <- family_col
      sym_unique <- -(64+seq_along(family_unique))
      for (k in seq_along(family_unique)){
        change <- (sym_name == family_unique[k])
        sym_var[change] <- -64-k  #use UNICODE plotting symbols (mostly capital letters)
      }
    }
    
    print(paste("Working on graphs by ", classification, sep=""))
    
    #O2 consumption box-and-whisker plot
    file_name <- paste(classification," O2 consumption.pdf",sep="")
    pdf(file=file_name, width=5.30, height=4.500)
    par <- op
    par(mai=c(2.0,0.82,0.5,0.4))
    abc <- boxplot(log10(input_data[,O2_col])~input_data[,data_col],
                   xlab="", ylab="log10(O2 consumption)", xaxt = "n", 
                   main=paste("O2 consumption by", classification)) 
    axis(side=1, las=2, at=seq_along(abc$names), labels=abc$names, xpd=NA, cex.axis=0.7)
    dev.off()
    
    #Eigenvalue plots
    file_name <- paste(classification," ",stats_name," eig.pdf",sep="")
    pdf(file=file_name, width=5.30, height=4.500)
    par <- op
    abc <- barplot(height=gen_pca$eig[1:10,2], 
                   names.arg=paste("PC ",1:10, sep=""), cex.names=0.7,
                   xlab="PC", ylab="% Explained By PC Variance", 
                   main=paste(stats_name,"PC Variance"))
    ax_limits <- par("usr")
    delta_y <- ax_limits[4]-ax_limits[3]
    text(x=abc, y=gen_pca$eig[1:10,2]+0.01*delta_y, adj=c(0.5,0), 
         labels=round(gen_pca$eig[1:10,2], digits=2), xpd=NA, cex=0.8)
    dev.off()
    
    #3D (PC1 vs PC2 vs O2) consumption plot
    file_name <- paste(classification," ",stats_name," 3D (PC1, PC2, O2).pdf",sep="")
    pdf(file=file_name, width=5.30, height=4.500)
    par <- op
    par(mai=c(1.02,0.82,0.82,1.5))
    points3D(x=gen_pca$ind$coord[,1], y=gen_pca$ind$coord[,2], z=log10(input_data[,O2_col]),
             xlab="PC 1", ylab="PC 2", zlab="log10(O2 consumed per cell)", 
             cex.axis=0.6, cex.main=0.7, cex.lab=0.6,
             main=paste(classification,stats_name,"PC1 vs. PC2 vs. O2"),
             pch=sym_var, 
             col=viridis(401), colvar=log10(input_data[,O2_col]), 
             colkey=list(side=4, length=1, width=1, cex.axis=0.8, dist=-0.02))
    ax_limits <- par("usr")
    legend(x=ax_limits[2]+0.275*(ax_limits[2]-ax_limits[1]), 
           y=ax_limits[4]+0.01*(ax_limits[4]-ax_limits[3]), 
           legend=data_unique, 
           pch=sym_unique, col="black", cex=0.5, xpd=NA)
    dev.off()
    
    #PC1 vs PC2 plot
    file_name <- paste(classification," ",stats_name," PC1 vs PC2.pdf",sep="")
    pdf(file=file_name, width=5.30, height=4.500)
    par <- op
    par(mai=c(1.02,0.82,0.82,2.3))
    points2D(x=gen_pca$ind$coord[,1], y=gen_pca$ind$coord[,2], type="p",
             xlab="PC 1", ylab="PC 2", 
             main=paste(classification,stats_name,"PC1 vs. PC2"),
             pch=sym_var, 
             col=viridis(401), colvar=log10(input_data[,O2_col]), colkey=TRUE)
    ax_limits <- par("usr")
    legend(x=ax_limits[2]+0.5*(ax_limits[2]-ax_limits[1]), 
           y=ax_limits[4]+0.1*(ax_limits[4]-ax_limits[3]), 
           legend=data_unique, 
           pch=sym_unique, col="black", cex=0.7, xpd=NA)
    dev.off()
    
    for (i in 1:5){
      file_name <- paste(classification," ",stats_name," PC",i," vs O2.pdf",sep="")
      pdf(file=file_name, width=5.30, height=4.500)
      par <- op
      par(mai=c(1.02,0.82,0.82,2.3))
      points2D(x=gen_pca$ind$coord[,i], y=log10(input_data[,O2_col]),
               xlab=paste("PC",i,sep=""), ylab="log10(O2 consumed per cell)", 
               main=paste(classification," ",stats_name," PC",i," vs O2", sep=""),
               pch=sym_var, 
               col=viridis(401), colvar=log10(input_data[,O2_col]), 
               colkey=TRUE)
      ax_limits <- par("usr")
      legend(x=ax_limits[2]+0.5*(ax_limits[2]-ax_limits[1]), 
             y=ax_limits[4]+0.1*(ax_limits[4]-ax_limits[3]), 
             legend=data_unique, 
             pch=sym_unique, col="black", cex=0.7, xpd=NA)
      dev.off()
      
      #PC vs contributions plot
      pdf(file=paste(classification," ",stats_name," PC",i,".pdf",sep=""), 
          width=5.30, height=4.500)
      contribs <- gen_pca$var$contrib
      sort1 <- order(-contribs[,i])
      ht <- contribs[sort1[1:10],i]
      gen_label <- row.names(contribs[sort1[1:10],])
      par <- op
      par(oma=c(3.5,0,0,0))
      abc <- barplot(height=ht, ylab="% Contributed to PC",i, names.arg="", 
                     main=paste(classification," ",stats_name," PC",i,sep=""))
      ax_limits <- par("usr")
      delta_y <- ax_limits[4]-ax_limits[3]
      text(x=0.5,y=seq(from=ax_limits[3]-0.025*delta_y, 
                       to=ax_limits[3]-0.35*delta_y, 
                       length.out=10), 
           labels=gen_label, xpd=NA, cex=0.5, adj=c(0,0.5))
      text(x=abc, y=ht+0.01*delta_y, adj=c(0.5,0), 
           labels=round(ht, digits=3), xpd=NA, cex=0.8)
      dev.off()
    }
  }
  par <- op
}